In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.17


### Load Workspace

In [6]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config(auth=azureml.core.authentication.InteractiveLoginAuthentication(force=True, tenant_id='72f988bf-86f1-41af-91ab-2d7cd011db47'))
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.
Found the config file in: C:\Users\makayser\Desktop\git\mxk_retinanet\notebook\aml_config\config.json
mak-ml	westeurope	makshared	westeurope


### Create Experiment

In [7]:
experiment_name = 'mxk-train'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create&Attach Compute

In [9]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = "gpucluster"
compute_min_nodes = 0
compute_max_nodes = 3
vm_size = "STANDARD_NC6"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating

ConnectionError: HTTPSConnectionPool(host='management.azure.com', port=443): Max retries exceeded with url: /subscriptions/50324bce-875f-4a7b-9d3c-0e33679f5d72/resourceGroups/makshared/providers/Microsoft.MachineLearningServices/workspaces/mak-ml/computes/gpucluster?api-version=2018-11-19 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000019621E94828>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

### Create Cloud Data Store

In [8]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

# ds.upload(src_dir=data_folder, target_path='mxk', overwrite=True, show_progress=True)

AzureBlob makml9496683038 azureml-blobstore-43aa3424-3674-489b-808b-1e49daacf13c
